In [1]:
import json
from utils import *

config = load_config()
PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/processed")

In [12]:
import pandas as pd
d = pd.read_csv("../data/raw/discharge.csv")
print(d['text'][0])

 
Name:  ___                    Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
Anticholinergics,Other / Reglan
 
Attending: ___.
 
Chief Complaint:
Abdominal pain
 
Major Surgical or Invasive Procedure:
None

 
History of Present Illness:
Patient is a ___ yo woman with history of chronic pancreatitis 
s/p cholecystectomy and sphincterotomy who presents with 1wk of 
worsening abdominal pain. As per patient, the pain is 
intermittent, sharp and ___ in quality. It localizes to her 
mid/righ upper abdomen and radiates up the chest wall and to the 
back. Patient finds this pain to be very similar to her prior 
pancreatitis flare-ups. Denies precipitants including alcohol 
use, abd. trauma, infections or h/o gallstones. She notes that 
she is on a restricted diet (no caffeine/fatty food/fried 
food/dairy) as part of her pancreatitis management. Also notes 
that pain worsens with meals; and 

In [2]:
def load_discharge_data() :
    discharge_path = DATA_PATH.joinpath("discharge_dataset.json")
    with open(discharge_path, 'r') as f :
        data = json.load(f)
    return data

def save_discharge_data(discharge) :
    discharge_path = DATA_PATH.joinpath("discharge_dataset.json")
    with open(discharge_path, 'w') as f :
        json.dump(discharge, f)
    

In [3]:
discharge = load_discharge_data()
discharge = discharge[:10000]

In [4]:
print(discharge[0]['instruction'])

You are a helpful assistant, an expert in medical domain. 
Extract top {topN} main diagnosis/symptoms or conditions mentioned in the medical note. 
Following the diagnosis/symptoms or conditions, identify the medical tests related to it.
If there isn't any medical tests related to it, just start listing the next important diagnosis/symptoms or conditions.
If there are no additional diagnosis/symptoms or conditions that you can identify, just list the existing ones and finalize the output. 
Don't write no symptoms, or any indication that there is no other diagnosis/symptoms or conditions.
Do not modify or abbreviate what is written in the notes. Just extract them as they are.
Make sure the highest priority is assigned with a smaller number.
We give you an example, do follow as below.
The format should be as follows

1. key symptom, lab tests, condition
2. key symptom, lab tests, condition
3. key symptom, lab tests, condition
4. key symptom, lab tests, condition
5. key symptom, lab tests

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(config.model_path("mistral7b"))

In [9]:
def format_texts(sample) :
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:").format_map(sample)
    

In [13]:
discharge = list(map(format_texts, discharge))

In [15]:
max_val = 0
for text in discharge :
    tokenized_text = tokenizer.tokenize(text)
    if max_val < len(tokenized_text) :
        max_val = len(tokenized_text)

print(max_val)

15659


In [34]:
save_discharge_data(discharge)